<a href="https://colab.research.google.com/github/malaxovakatyunya/python-ai-Malahova-Ekaterina/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о статуях мира

**Цель**: Научиться читать CSV-файлы из репозитория GitHub в Google Colab и выполнять базовую проверку данных с помощью pandas.

**Данные:**
- `World_Statues.csv` — информация о памятниках и статуях мира: название, тип, высота, год создания, страна и материал

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл в pandas DataFrame
3. Очищаем и анализируем структуру столбцов
4. Просматриваем первые строки и выполняем быструю валидацию данных

## 🐱 [1] Клонируем репозиторий курса в Colab

In [2]:
# 🐱 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai-Malahova-Ekaterina"):
    !git clone -q https://github.com/malaxovakatyunya/python-ai-Malahova-Ekaterina.git

%cd python-ai-Malahova-Ekaterina

print("✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina")

/content/python-ai-Malahova-Ekaterina/python-ai-Malahova-Ekaterina
✅ Репозиторий готов, теперь мы работаем внутри папки python-ai-Malahova-Ekaterina


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [27]:
# 🐱 Шаг 2A. Чтение CSV-файлов в pandas

import pandas as pd

df_statues = pd.read_csv("data/World_Statues.csv")

print("✅ Загружено строк в df_cost:", len(df_statues))


✅ Загружено строк в df_cost: 1567


## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбцы `statue` и `statueType` содержат URL Wikidata (ссылки на объекты) — они не нужны для анализа.
- Столбцы `statueLabel`, `statueTypeLabel`, `countryLabel`, `materialLabel` содержат читаемые подписи (название статуи, тип, страна, материал).

В этом шаге мы:
- удалим столбцы с URL Wikidata (`statue`, `statueType`);
- переименуем столбцы с суффиксом `Label`, убрав его:
  - `statueLabel` → `statue`
  - `statueTypeLabel` → `statueType`
  - `countryLabel` → `country`
  - `materialLabel` → `material`
- приведём числовые столбцы (`height`, `creationYear`) к корректным типам:
  - `height` → `float` (высота может содержать дробную часть)
  - `creationYear` → `int` (год — целое число)

При приведении к числам мы используем:
- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0;
- `astype(...)` — переводит столбец к нужному типу (`float` или `int`).

> ⚠️ **Важно:** этот шаг обязателен, чтобы избавиться от технических идентификаторов Викиданных и получить удобную таблицу для анализа статуй.

In [28]:
# 🧹 Шаг 2B. Очистка и переименование столбцов

# Удаляем столбцы с URL Wikidata
df_statues = df_stat.drop(columns=["statue", "statueType"])

# Переименовываем столбцы с суффиксом Label
df_statues = df_stat.rename(columns={
    "statueLabel": "statue",
    "statueTypeLabel": "statueType",
    "countryLabel": "country",
    "materialLabel": "material"
})

# Приводим числовые столбцы к корректным типам
df_statues["height"] = pd.to_numeric(
    df_statues["height"], errors="coerce"
).fillna(0).astype(float)

df_statues["creationYear"] = pd.to_numeric(
    df_statues["creationYear"], errors="coerce"
).fillna(0).astype(int)

print("✅ Данные очищены и готовы к анализу")
print("\n📋 Итоговая структура датасета:")
print(df_statues.dtypes)
display(df_stat.head(5))

✅ Данные очищены и готовы к анализу

📋 Итоговая структура датасета:
statue           object
statue           object
statueType       object
statueType       object
height          float64
creationYear      int64
country          object
material         object
dtype: object


,statue,statueLabel,statueType,statueTypeLabel,height,creationYear,countryLabel,materialLabel
0,http://www.wikidata.org/entity/Q646012,Памятник Нидервальд,http://www.wikidata.org/entity/Q575759,военный мемориал,38.18,1883.0,Германия,камень
1,http://www.wikidata.org/entity/Q646012,Памятник Нидервальд,http://www.wikidata.org/entity/Q575759,военный мемориал,38.18,1883.0,Германия,бронза
2,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q29168169,колоссальная статуя Иисуса Христа,8.00,1920.0,Бразилия,железобетон
3,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q29168169,колоссальная статуя Иисуса Христа,8.00,1920.0,Бразилия,талькохлорит
4,http://www.wikidata.org/entity/Q79961,статуя Христа-Искупителя,http://www.wikidata.org/entity/Q179700,статуя,30.10,1920.0,Бразилия,железобетон


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор DataFrame со статуями:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по высоте (`height`) и году создания (`creationYear`).

Для удобства используем функцию `show_info(df, name)`, чтобы компактно вывести информацию о датасете.

In [29]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных

show_info(df_statues, "Статуи мира (df_statues)", n=10)  # Показываем 10 строк для лучшего обзора разнообразия

print("\n📈 Статистика по высоте (height, м):")
print(df_statues["height"].describe())

print("\n📅 Статистика по году создания (creationYear):")
print(df_statues["creationYear"].describe())

print("\n🌍 Топ-10 стран по количеству статуй:")
print(df_statues["country"].value_counts().head(10))


📊 Статуи мира (df_statues)
Размер: (1567, 8)
Столбцы: statue, statue, statueType, statueType, height, creationYear, country, material

Первые строки:
                                   statue                    statue  \
0  http://www.wikidata.org/entity/Q646012       Памятник Нидервальд   
1  http://www.wikidata.org/entity/Q646012       Памятник Нидервальд   
2   http://www.wikidata.org/entity/Q79961  статуя Христа-Искупителя   
3   http://www.wikidata.org/entity/Q79961  статуя Христа-Искупителя   
4   http://www.wikidata.org/entity/Q79961  статуя Христа-Искупителя   
5   http://www.wikidata.org/entity/Q79961  статуя Христа-Искупителя   
6  http://www.wikidata.org/entity/Q805445    Christ the King statue   
7  http://www.wikidata.org/entity/Q805445    Christ the King statue   
8  http://www.wikidata.org/entity/Q805445    Christ the King statue   
9  http://www.wikidata.org/entity/Q805445    Christ the King statue   

                                 statueType  \
0    http://www.wiki

## 🔬 [4] Глубокая проверка данных: эволюция материалов и национальные рекордсмены

Мы проведём два фундаментальных анализа, раскрывающих историю монументального искусства через призму данных:

### 1. Эволюция материалов по историческим эпохам
Как менялись технологии строительства статуй на протяжении двух с половиной тысячелетий?
- Разобьём все статуи на 5 исторических периодов (от античности до наших дней)
- Для каждой эпохи определим:
  - Топ-3 доминирующих материала
  - Среднюю высоту статуй для каждого материала
  - Технологический «прорыв» эпохи (например, переход от бронзы к железобетону)

> ⚠️ **Очистка данных:** перед анализом исключим аномалии (высота > 200 м). Самая высокая реальная статуя в мире — Статуя Единства (Индия) — имеет высоту 182 м. Запись с высотой 4800 м — ошибка Викиданных.

### 2. Национальные рекордсмены: самые высокие статуи мира
Какие страны создали самые грандиозные монументы?
- Для топ-15 стран по количеству статуй найдём абсолютного высотного рекордсмена
- Для каждой рекордной статуи укажем:
  - Название и тип монумента
  - Точную высоту в метрах
  - Год создания и материал
- Составим глобальный рейтинг самых высоких статуй по странам

Эти анализы превратят сырые данные в историю технологического прогресса и культурного соперничества наций в создании вечных символов.

In [38]:
# 🔬 Шаг 4. Глубокая проверка: материалы по эпохам + национальные рекордсмены

import pandas as pd

# ------------------------------------------------------------
# ПРЕДВАРИТЕЛЬНАЯ ОЧИСТКА: убираем аномалии по высоте (> 200 м)
# ------------------------------------------------------------
df_clean = df_statues[df_statues["height"] <= 200].copy()
anomalies_removed = len(df_statues) - len(df_clean)
print(f"✅ Удалено аномальных записей (высота > 200 м): {anomalies_removed}")
print(f"📊 Записей для анализа: {len(df_clean)}\n")

# ------------------------------------------------------------
# 1. ЭВОЛЮЦИЯ МАТЕРИАЛОВ ПО ИСТОРИЧЕСКИМ ЭПОХАМ
# ------------------------------------------------------------
print("="*80)
print("1️⃣  ЭВОЛЮЦИЯ МАТЕРИАЛОВ В МОНОУМЕНТАЛЬНОМ ИСКУССТВЕ ПО ЭПОХАМ")
print("="*80)

# Функция определения эпохи по году
def get_epoch(year):
    if year < 0:
        return "Античность (до н.э.)"
    elif 0 <= year < 1500:
        return "Средневековье (0–1499)"
    elif 1500 <= year < 1800:
        return "Эпоха Просвещения (1500–1799)"
    elif 1800 <= year < 1950:
        return "Индустриальная эра (1800–1949)"
    else:
        return "Современность (1950–н.в.)"

df_clean["epoch"] = df_clean["creationYear"].apply(get_epoch)

# Порядок эпох для корректной сортировки
epoch_order = [
    "Античность (до н.э.)",
    "Средневековье (0–1499)",
    "Эпоха Просвещения (1500–1799)",
    "Индустриальная эра (1800–1949)",
    "Современность (1950–н.в.)"
]

# Анализ для каждой эпохи
for epoch_name in epoch_order:
    epoch_data = df_clean[df_clean["epoch"] == epoch_name]
    total = len(epoch_data)

    if total == 0:
        continue

    print(f"\n{'─'*80}")
    print(f"🔹 {epoch_name} | Всего статуй: {total}")
    print(f"{'─'*80}")

    # Топ-3 материала с количеством и средней высотой
    material_stats = (
        epoch_data.groupby("material")
        .agg(
            count=("material", "size"),
            avg_height=("height", "mean"),
            min_height=("height", "min"),
            max_height=("height", "max")
        )
        .sort_values("count", ascending=False)
        .head(3)
    )

    for idx, (material, row) in enumerate(material_stats.iterrows(), 1):
        print(
            f"{idx}. {material:25s} | "
            f"{int(row['count']):3d} шт. | "
            f"ср. высота: {row['avg_height']:5.1f} м | "
            f"диапазон: {row['min_height']:.1f}–{row['max_height']:.1f} м"
        )

# ------------------------------------------------------------
# 2. НАЦИОНАЛЬНЫЕ РЕКОРДСМЕНЫ: САМЫЕ ВЫСОКИЕ СТАТУИ ПО СТРАНАМ
# ------------------------------------------------------------
print("\n" + "="*80)
print("2️⃣  НАЦИОНАЛЬНЫЕ РЕКОРДСМЕНЫ: САМЫЕ ВЫСОКИЕ СТАТУИ МИРА")
print("="*80)

# Берём топ-15 стран по количеству статуй
top_countries = df_clean["country"].value_counts().head(15).index.tolist()

# Собираем рекордсменов
records = []
for country in top_countries:
    country_data = df_clean[df_clean["country"] == country]
    max_idx = country_data["height"].idxmax()
    max_row = country_data.loc[max_idx]

    records.append({
        "Страна": country,
        "Название": max_row["statue"],
        "Тип": max_row["statueType"],
        "Высота (м)": max_row["height"],
        "Год": int(max_row["creationYear"]),
        "Материал": max_row["material"]
    })

df_records = pd.DataFrame(records)

# Сортируем по высоте (от самых высоких)
df_records = df_records.sort_values("Высота (м)", ascending=False).reset_index(drop=True)
df_records.index = df_records.index + 1  # Нумерация с 1

# Форматируем таблицу для красивого вывода
print("\n🏆 Глобальный рейтинг самых высоких статуй по странам:")
print("   (топ-15 стран по количеству монументов)\n")

# Выводим вручную для контроля форматирования
print(f"{'Место':<6} {'Страна':<15} {'Высота':<10} {'Год':<6} {'Материал':<20} {'Название'}")
print("-" * 100)

for idx,

# ------------------------------------------------------------
# КРАТКИЙ ИТОГ
# ------------------------------------------------------------
print("\n" + "="*70)
print("💡 КЛЮЧЕВЫЕ ИНСАЙТЫ")
print("="*70)
print(f"• Самая древняя статуя в датасете: {oldest_table.iloc[0]['Название']} "
      f"({oldest_table.iloc[0]['Год создания']}, {oldest_table.iloc[0]['Страна']})")
print(f"• Самая высокая статуя после очистки: {df_records.iloc[0]['Название статуи']} "
      f"({df_records.iloc[0]['Высота (м)']} м, {df_records.iloc[0]['Страна']})")
print(f"• Самый необычный материал в древности: {oldest_table.iloc[0]['Материал']}")
print(f"• Прорыв железобетона: доминирует в эпоху «Современность (1900–н.в.)»")

SyntaxError: invalid syntax (ipython-input-3999917246.py, line 117)

## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий GitHub в Colab
- ✅ Прочитали 2 CSV-файла из `data/examples/`
- ✅ Удалили URL Wikidata и переименовали столбцы (`*Label → короткие имена`)
- ✅ Проверили структуру данных (размер, столбцы, первые строки)
- ✅ Посмотрели базовую статистику по бюджету (`capital_cost`)
- ✅ Выполнили быструю валидацию:
  - количество уникальных фильмов, стран, жанров
  - диапазоны значений
  - топ стран и жанров по числу записей

Теперь у нас есть **аккуратные, проверенные таблицы**, с которыми удобно работать дальше.

В отдельном ноутбуке для следующей недели мы будем использовать **те же данные** для:
- более сложного анализа (группировки, фильтрация),
- и построения визуализаций (графики и диаграммы). 🎨